Google Colab Setup

In [24]:
!git clone https://github.com/jan1na/Bleeding-Detecting-on-Capsule-Endoscopy.git

%cd Bleeding-Detecting-on-Capsule-Endoscopy
!git checkout "janina"

import sys
sys.path.append('/content/Bleeding-Detecting-on-Capsule-Endoscopy/scripts')

Cloning into 'Bleeding-Detecting-on-Capsule-Endoscopy'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 70 (delta 36), reused 43 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (70/70), 4.78 MiB | 36.23 MiB/s, done.
Resolving deltas: 100% (36/36), done.
/content/Bleeding-Detecting-on-Capsule-Endoscopy/Bleeding-Detecting-on-Capsule-Endoscopy/Bleeding-Detecting-on-Capsule-Endoscopy/Bleeding-Detecting-on-Capsule-Endoscopy/Bleeding-Detecting-on-Capsule-Endoscopy
Branch 'janina' set up to track remote branch 'janina' from 'origin'.
Switched to a new branch 'janina'


Drive Setup

In [25]:
from google.colab import drive

# Mount Google Drive (for data storage)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Deep Learning Part

## TODO
* losses should be weighted for different classes, there are 8 to 9 times of healthy images than there is for bleeding, so model will be inclined to predict healthy
* hyperparameters for deep learning runs should be saved

## Imports

In [26]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime
import itertools

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
from torch.optim import Adam, lr_scheduler
from torch.utils.data.sampler import WeightedRandomSampler

from models import MobileNetV2, GoogleNet, ResNet, AlexNet, VGG19
from bleeding_dataset import BleedDataset

## Training Space

### Hyperparameters

In [27]:
SAVE_PATH = "/content/drive/MyDrive/Colab Notebooks/DL4MI/runs/"

TRAIN_TEST_SPLIT = (0.8, 0.1) # remaining parts will be test
DIRECTORY_PATH = "/content/drive/MyDrive/Colab Notebooks/DL4MI/project_capsule_dataset"
BATCH_SIZE = 32
LR = 0.001 # learning rate

NUM_OF_EPOCHS = 20
EARLY_STOP_LIMIT = 3

THRESHOLD = 0.5 # predictions bigger than threshold will be counted as bleeding prediction, and lower ones will be healthy prediction
MODEL =  ResNet
AUGMENT_TIMES = 8

### Dataset, Model etc. Inıtıalization

In [28]:
### ---|---|---|---|---|---|---|---|---|---|--- MODEL & DATASET ---|---|---|---|---|---|---|---|---|---|--- ###
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()

# Model Initialization
def initialize_model(model_class, save_path):
    model = model_class().to(device)
    model_serial_number = f"training_with_{model.__class__.__name__}_{datetime.now().strftime('on_%m.%d._at_%H:%M:%S')}"
    model_serial_path = os.path.join(save_path, model_serial_number)
    os.makedirs(model_serial_path, exist_ok=True)
    return model, model_serial_path

model, model_serial_path = initialize_model(MODEL, SAVE_PATH)

# Dataset Preparation
def prepare_datasets(dataset_class, directory, split_ratios, batch_size, image_mode="RGB", seed=0):
    full_dataset = dataset_class(directory, mode=image_mode, apply_augmentation=False)
    total_size = len(full_dataset)
    train_size = int(split_ratios[0] * total_size)
    test_size = int(split_ratios[1] * total_size)
    validation_size = total_size - train_size - test_size

    torch.manual_seed(seed)
    train_dataset, validation_dataset, test_dataset = random_split(full_dataset, [train_size, validation_size, test_size])

    # Apply augmentation **only to the training set**
    train_dataset.dataset.enable_augmentation(augment_times=AUGMENT_TIMES)

    # Oversampling for bleeding images
    labels = [label for _, label in train_dataset.dataset.data]
    class_counts = np.bincount(labels)  # Count occurrences per class
    class_weights = 1.0 / class_counts  # Inverse class frequency

    weights = [class_weights[label] for _, label in train_dataset.dataset.data]
    sampler = WeightedRandomSampler(weights, len(weights))

    return {
        "train": DataLoader(train_dataset, batch_size=batch_size, num_workers=4, pin_memory=True, sampler=sampler),
        "validation": DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True),
        "test": DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True),
    }

data_loaders = prepare_datasets(BleedDataset, DIRECTORY_PATH, TRAIN_TEST_SPLIT, BATCH_SIZE, image_mode="RGB", seed=0)

# Penalty for imbalanced dataset
bleeding_weight = 6161 / (713 * AUGMENT_TIMES)
weights = torch.tensor([1.0, bleeding_weight]).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=weights[1])

#criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
# scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_OF_EPOCHS)


### Training Loop

In [29]:
### ---|---|---|---|---|---|---|---|---|---|--- TRAINING ---|---|---|---|---|---|---|---|---|---|--- ###
train_losses, validation_losses = [], []
min_validation_loss = None # minimum achieved loss on validation dataset, used for early stopping
min_validation_path = None # path to model checkpoint file
early_stop_step = 0

for epoch in range(NUM_OF_EPOCHS):
    averaged_training_loss = 0
    for batch_idx, (images, labels) in tqdm(enumerate(data_loaders['train']), leave=False):
        images, labels = images.to(device), labels.to(device)

        model = model.train()
        outputs = model(images.type(torch.float))

        float_outputs = outputs[:,0].type(torch.float)
        float_labels = labels.type(torch.float)
        train_loss = criterion(float_outputs, float_labels)
        averaged_training_loss = averaged_training_loss + train_loss

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

    # calculating the average loss in this epoch's training loop
    averaged_training_loss = averaged_training_loss / len(data_loaders['train'])

    with torch.no_grad():
        model = model.eval()

        # calculate validation loss
        validation_loss = 0.0
        for validation_images, validation_labels in data_loaders['validation']:
            validation_images, validation_labels = validation_images.to(device), validation_labels.to(device)

            validation_outputs = model(validation_images.type(torch.float))

            float_validation_outputs = validation_outputs[:,0].type(torch.float)
            float_validation_labels = validation_labels.type(torch.float)
            validation_loss += criterion(float_validation_outputs, float_validation_labels).item()

        # and average the loss over dataset length
        validation_loss /= len(data_loaders['validation'])

    # if this is first validation or a new minimum is achieved
    if min_validation_loss is None or validation_loss < min_validation_loss:
        early_stop_step = 0
        min_validation_loss = validation_loss

        # if there is a checkpoint file, remove it
        if min_validation_path is not None:
            os.remove(min_validation_path)

        # save the new checkpoint file
        min_validation_path = os.path.join(model_serial_path, "min_validation_loss:"+str(min_validation_loss) + "_epoch:" + str(epoch) + ".pth")
        torch.save(model, min_validation_path)
    else:
        early_stop_step += 1

    # log the losses, and append to the lists
    print(f"Epoch: {epoch+1} | training loss: {averaged_training_loss.item()} | min validation loss: {min_validation_loss}", flush=True)
    train_losses.append(averaged_training_loss.item())
    validation_losses.append(validation_loss)
    scheduler.step()

    # check for early stopping
    if early_stop_step >= EARLY_STOP_LIMIT:
        print("early stopping...")
        break

IndexError: Caught IndexError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 50, in fetch
    data = self.dataset.__getitems__(possibly_batched_index)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataset.py", line 420, in __getitems__
    return [self.dataset[self.indices[idx]] for idx in indices]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataset.py", line 420, in <listcomp>
    return [self.dataset[self.indices[idx]] for idx in indices]
                         ~~~~~~~~~~~~^^^^^
IndexError: list ind

In [ ]:
plt.plot(train_losses, color='blue', label='Train Loss')
plt.plot(validation_losses, color='orange', label='Validation Loss')
plt.legend()
plt.savefig(os.path.join(model_serial_path, "losses.png"))

## Testing Space

In [ ]:
### ---|---|---|---|---|---|---|---|---|---|--- TESTING ---|---|---|---|---|---|---|---|---|---|--- ###
loaded_model = model.__class__().to(device)
loaded_model.load_state_dict(torch.load(min_validation_path))
loaded_model = loaded_model.eval()

# class_correct counts how many correct predictions for that label [corrects_for_label_0, corrects_for_label_1]
# class_total counts how many predictions are there for that label [predictions_for_label_0, predictions_for_label_1]
class_correct, class_total = [0,0], [0,0]
with torch.no_grad():
    for test_images, test_labels in tqdm(data_loaders['test']):
        test_images, test_labels = test_images.to(device), test_labels.to(device)

        test_outputs = loaded_model(test_images.type(torch.float))

        test_outputs = test_outputs.squeeze().type(torch.float)
        test_outputs[test_outputs >= THRESHOLD] = 1
        test_outputs[test_outputs < THRESHOLD] = 0

        # calculate indices for correct predictions
        correct = (test_outputs == test_labels).squeeze()
        for e, label in enumerate(test_labels):
            # increase the correct prediction count for that label
            class_correct[label] += correct[e].item()
            class_total[label] += 1

In [ ]:
# Total: accuracy for whole dataset
print(f"Total accuracy: {sum(class_correct)/sum(class_total)} on threshold: {THRESHOLD}")
print(f"Healthy detection: {class_correct[0]}/{class_total[0]} | accuracy: {class_correct[0]/class_total[0]}")
print(f"Bleeding detection: {class_correct[1]}/{class_total[1]} | accuracy: {class_correct[1]/class_total[1]}")

with open(os.path.join(model_serial_path, "accuracy.txt"), 'w') as txt:
    txt.write(f"Total accuracy: {sum(class_correct)/sum(class_total)} on threshold: {THRESHOLD}\n")
    txt.write(f"Healthy detection: {class_correct[0]}/{class_total[0]} | accuracy: {class_correct[0]/class_total[0]}\n")
    txt.write(f"Bleeding detection: {class_correct[1]}/{class_total[1]} | accuracy: {class_correct[1]/class_total[1]}\n")

torch.cuda.empty_cache()